# LOCALIZACIONES DEFINIDAS
posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
importar librerias

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

# FUNCIONES DE LLAMADA A LA API

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [35]:
# Asignar todos los token a una variable
tk_id = os.getenv("token_client_id")

In [36]:
tk_secret = os.getenv("token_client_secret")

In [37]:
git_tk = os.getenv("github_tk")

In [38]:
# Asignar una variable a la url
url = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
#pyjsonviewer.view_data(json_data = LaVariableQueQuierasSinComillas)

In [11]:
# A esta función entra una url, unas coordenadas y lo que queremos que nos devuelva la API(query).
# Nos devuelve un lista de diccionarios de sitios con esa descripción cerca de las coordenadas que hemos puesto.
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response

In [12]:
# Esta función se mete dentro del la lista de diccionarios previamente creada y nos devuelve todas las variavbles 
# que queremos en una lista de listas. En este caso he escogido el nombre, el tipo de negocio, la latitud, la 
# longitud el país y la dirección 

def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    cc = []
    adress = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        cc.append(i["venue"]["location"]["cc"])
    for i in response:
        adress.append(i["venue"]["location"]["formattedAddress"][0])      
    response2 = [name , type_, cc, adress,  lat, lng ]

    return response2

In [40]:
# Esta función crea un dataframe y lo transpone cambiando las filas por columnas. Además le da un nombre específico
# a cada columna en base a lo que esa columna muestra.
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Country", 3:"Adress", 4:"Latitude", 5:"Longitude", })
    return df1

In [14]:
# Está función junta los valores de latitud y longitud en un punto para que geopandas lo entienda.
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [41]:
# Esta función le añade cosas a la función anterior para que la localización quede de una manera clara para el 
# ordenador
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [42]:
# Esta función junta todas las funciones anteriores y crea una única función para crear un dataframe claro.
def df_final1(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["location"] = get_loc(d)
    df_terminado = d[["Name","Type","Country", "Adress","Latitude","Longitude","location"]]
    return df_terminado

# FUNCTIONS TO GET CONDITIONS OF A LOCATION

# SCHOOLS

In [17]:
# Esta función llama a la API para ver todos los colegios que hay cuando le pasas una ubicación, los colegios que
# que busca son los colegios dentro de la lista de colegios. Aparte, añade una columna llamada classification
# al dataframe para saber que son todo colegios.
Schools = ["Nursery School","Elementary School","Middle School","High School"]
def get_schools(location, Schools):
    Nursery_sch = df_final1(url, location, Schools[0])
    Elementary_sch = df_final1(url, location, Schools[1])
    Middle_sch = df_final1(url, location, Schools[2])
    High_sch = df_final1(url, location, Schools[3])
    all_schools_bad_index = pd.concat([Nursery_sch, Elementary_sch, Middle_sch, High_sch])
    all_schools = all_schools_bad_index.reset_index(drop=True)
    all_schools.insert(1, "Classification", "School")
    return all_schools

# DESIGNING COMPANIES

In [18]:
#Esta función llama a la API para ver todos los estudios de diseño que hay cuando le pasas una ubicación, aparte 
# tambien mete una columna nueva para aclarar que son estudios de diseño ya que la APi te clasifica lo que te
# devuelve con otras cosas.
def get_design_studio(location):
    DesignCo = df_final1(url, location, "Design Studio")
    DesignCo.insert(1, "Classification", "Designing Studios")
    return DesignCo


# TECH STARTUPS

In [19]:
#Esta función llama a la API para ver todos las start ups tecnológicas que hay cuando le pasas una ubicación.
# También añade una columna de clasificación para definirlas como Technological start ups.
def get_tech_startups(location):
    TechStaUp = df_final1(url, location, "Tech Startup")
    TechStaUp.insert(1, "Classification", "Technological Startups")
    return TechStaUp

    

# LEISURE

In [20]:
# A esta función le pasas una localización y la palabra leisure. Entonces te devuelve todas las discotecas y sitios
# de ocio cerca de la localización establecida. Además tambien añade una columna clasificando todos esos sitios como
# Leisure.

leisure = ["Nightclub", "Piano Bar", "Pub", "Irish Pub"]
def get_leisure(location, leisure):
    Disco = df_final1(url, location, leisure[0])
    Piano_bar = df_final1(url, location, leisure[1])
    Pub = df_final1(url, location, leisure[2])
    irish_pub = df_final1(url, location, leisure[3])
    all_leisure_bad_index = pd.concat([Disco, Piano_bar, Pub, irish_pub])
    leisure = all_leisure_bad_index.reset_index(drop=True)
    leisure.insert(1, "Classification", "Leisure Places")
    return leisure

# TRAVELLING STATIONS

In [21]:
# Esta función recibe una url y la palabra stations. Y te devuelve un dataframe con todas las estaciones de bus,
# de metro y de tren cerca de tu localización. También clasifica todas estas estaciones en una columna, llamda 
# etaciones.
stations = ["Metro Station", "Light Rail Station", "Bus Station"  ]
def get_closeby_stations(location, stations):
    Metro = df_final1(url, location, stations[0])
    Rail_Station = df_final1(url, location, stations[1])
    Bus_Station = df_final1(url, location, stations[1])
    all_stations_bad_index = pd.concat([Metro, Rail_Station, Bus_Station])
    stations = all_stations_bad_index.reset_index(drop=True)
    stations.insert(1, "Classification", "Travelling Stations")
    return stations

# PARKS

In [22]:
# A esta función le pasas una localización y te devuelve todos los parque cercanos. Además tambien incluye una 
# nueva colmna que clasifica a todas las variables como parques
def get_parks(location):
    Parks = df_final1(url, location, "Park")
    Parks.insert(1, "Classification", "Parks")
    return Parks

# DATAFRAME GLOBAL Y LIMPIO

In [23]:
# A esta función le pasas una localización y te devuelve la un dataframe global con todo lo previamente calculado 
#con varias columnas.
def df_FINAL(location):
    a = get_schools(location, Schools)
    b = get_design_studio(location)
    c = get_tech_startups(location)
    d = get_leisure(location, leisure)
    e = get_closeby_stations(location,stations)
    f = get_parks(location)
    global_dataframe_bad_index = pd.concat([a,b,c,d,e,f])
    global_df_final = global_dataframe_bad_index.reset_index(drop=True)
    return global_df_final

In [24]:
a = df_FINAL(london_loc)

In [25]:
a.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
71,Chambers Judd,Designing Studios,Design Studio,GB,372 Old Street,51.527102,-0.079329,"{'type': 'Point', 'coordinates': (51.527102092..."
202,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': (51.521708909..."
336,Smithfield Rotunda Garden,Parks,Park,GB,W Smithfield,51.518491,-0.101066,"{'type': 'Point', 'coordinates': (51.518490677..."
251,Hoxton Cabin,Leisure Places,Pub,GB,"132 Kingsland Rd, London E28DP",51.530647,-0.07695,"{'type': 'Point', 'coordinates': (51.530647165..."
210,The Crosse Keys (Wetherspoon),Leisure Places,Pub,GB,9 Gracechurch St.,51.512685,-0.084751,"{'type': 'Point', 'coordinates': (51.512684501..."
127,OpenGamma HQ,Technological Startups,Tech Startup,GB,185 Park Street,51.513577,-0.080466,"{'type': 'Point', 'coordinates': (51.513576767..."
105,bb (Brilliant Basics),Technological Startups,Tech Startup,GB,Zetland house (Scrutton St),51.523379,-0.083471,"{'type': 'Point', 'coordinates': (51.523379149..."
107,Ostmodern,Technological Startups,Tech Startup,GB,26 Leonard Street,51.524399,-0.086231,"{'type': 'Point', 'coordinates': (51.524398706..."
257,BrewDog Shoreditch,Leisure Places,Beer Bar,GB,51-55 Bethnal Green Rd,51.524599,-0.072573,"{'type': 'Point', 'coordinates': (51.524598942..."
133,Phantom,Technological Startups,Tech Startup,GB,Underwood St.,51.529934,-0.091208,"{'type': 'Point', 'coordinates': (51.529933929..."


# LONDON

In [26]:
london_schools = get_schools(london_loc, Schools)
london_designing_companies = get_design_studio(london_loc)
london_tech_startups = get_tech_startups(london_loc)
london_leisure = get_leisure(london_loc, leisure)
london_travelling_stations = get_closeby_stations(london_loc, stations)
london_parks = get_parks(london_loc)
london_df_entero = df_FINAL(london_loc)

In [27]:
# Importar todas las funciones de londres a un csx
london_schools.to_csv("Data/London/ln_schools.csv")
london_designing_companies.to_csv("Data/London/ln_desing.csv")
london_tech_startups.to_csv("Data/London/ln_tech_startups.csv")
london_leisure.to_csv("Data/London/ln_leisure.csv")
london_travelling_stations.to_csv("Data/London/ln_stations.csv")
london_parks.to_csv("Data/London/ln_parks.csv")
london_df_entero.to_csv("Data/London/ln_entero.csv")

In [28]:
london_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
259,The Minories,Leisure Places,Pub,GB,64–73 Minories,51.51083,-0.074882,"{'type': 'Point', 'coordinates': (51.5108301, ..."
334,Shepherdess Walk Park,Parks,Park,GB,Shepherdess Walk,51.532188,-0.093727,"{'type': 'Point', 'coordinates': (51.532188071..."
116,Inition,Technological Startups,Tech Startup,GB,23 Curtain Rd,51.522957,-0.080571,"{'type': 'Point', 'coordinates': (51.522956581..."
106,Bossa Studios,Technological Startups,Tech Startup,GB,Zetland House,51.523604,-0.083363,"{'type': 'Point', 'coordinates': (51.523604444..."
343,Royal Oak Court,Parks,Park,GB,United Kingdom,51.528508,-0.083596,"{'type': 'Point', 'coordinates': (51.528507919..."
63,Funky Howler Digital Design & Marketing Agency,Designing Studios,Design Studio,GB,"Kemp House, 152-160 City Road",51.527389,-0.089039,"{'type': 'Point', 'coordinates': (51.527389112..."
333,Ravenscroft Park,Parks,Park,GB,Columbia Rd.,51.529086,-0.071318,"{'type': 'Point', 'coordinates': (51.529085757..."
93,Lastminute print,Designing Studios,Design Studio,GB,29-31 Greatorex Street,51.518371,-0.067567,"{'type': 'Point', 'coordinates': (51.5183709, ..."
186,The George & Vulture,Leisure Places,Pub,GB,63 Pitfield St (at Haberdasher St),51.529294,-0.083685,"{'type': 'Point', 'coordinates': (51.529293822..."
312,Kingsway Tram Tunnel,Travelling Stations,Light Rail Station,GB,Kingsway,51.517659,-0.120455,"{'type': 'Point', 'coordinates': (51.517659081..."


# SINGAPORE

In [29]:
singapore_schools = get_schools(singapore_loc, Schools)
singapore_designing_companies = get_design_studio(singapore_loc)
singapore_tech_startups = get_tech_startups(singapore_loc)
singapore_leisure = get_leisure(singapore_loc, leisure)
singapore_travelling_stations = get_closeby_stations(singapore_loc, stations)
singapore_parks = get_parks(singapore_loc)
singapore_df_entero = df_FINAL(singapore_loc)

In [30]:
singapore_schools.to_csv("Data/Singapore/sg_schools.csv")
singapore_designing_companies.to_csv("Data/Singapore/sg_designco.csv")
singapore_tech_startups.to_csv("Data/Singapore/sg_techstar.csv")
singapore_leisure.to_csv("Data/Singapore/sg_leisure.csv")
singapore_travelling_stations.to_csv("Data/Singapore/sg_stations.csv")
singapore_parks.to_csv("Data/Singapore/sg_parks.csv")
singapore_df_entero.to_csv("Data/Singapore/sg_entero.csv")

In [31]:
singapore_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
349,NS Square (The Float @ Marina Bay),Parks,Event Space,SG,20 Raffles Ave,1.288167,103.858995,"{'type': 'Point', 'coordinates': (1.2881671823..."
293,Barossa,Leisure Places,Bar,SG,#01-01 Esplanade Mall (8 Raffles Avenue),1.289353,103.85605,"{'type': 'Point', 'coordinates': (1.2893531427..."
1,Little Skool House at SGH,School,Nursery School,SG,"Singapore General Hospital, Outram Road, SGH S...",1.277377,103.835714,"{'type': 'Point', 'coordinates': (1.2773766782..."
4,Wesley Child Development Centre,School,Nursery School,SG,Blk 29 telok blangah rise,1.27481,103.821335,"{'type': 'Point', 'coordinates': (1.2748100830..."
320,HarbourFront 6,Travelling Stations,Light Rail Station,SG,Singapore,1.279825,103.839092,"{'type': 'Point', 'coordinates': (1.279825, 10..."
295,Molly Malone's Irish Pub,Leisure Places,Bar,SG,56 Circular Road (Boat Quay),1.285895,103.84932,"{'type': 'Point', 'coordinates': (1.2858951708..."
55,Vanguard Building,Designing Studios,Design Studio,SG,71 - 77 Stamford Road,1.294825,103.849896,"{'type': 'Point', 'coordinates': (1.2948252250..."
124,Club illusion,Leisure Places,Nightclub,SG,Clarke Quay (Clarke Street),1.289533,103.845131,"{'type': 'Point', 'coordinates': (1.2895327212..."
350,Jubilee Park,Parks,Playground,SG,Fort Canning Park (Clemenceau Avenue / River V...,1.292984,103.845224,"{'type': 'Point', 'coordinates': (1.292984, 10..."
239,Purple Bar,Leisure Places,Pub,SG,Boat Quay,1.288617,103.849345,"{'type': 'Point', 'coordinates': (1.2886174554..."


# SAN FRANCISCO

In [32]:
sanfrancisco_schools = get_schools(san_francisco_loc, Schools)
sanfrancisco_designing_companies = get_design_studio(san_francisco_loc)
sanfrancisco_tech_startups = get_tech_startups(san_francisco_loc)
sanfrancisco_leisure = get_leisure(san_francisco_loc, leisure)
sanfrancisco_travelling_stations = get_closeby_stations(san_francisco_loc, stations)
sanfrancisco_parks = get_parks(san_francisco_loc)
sanfrancisco_df_entero = df_FINAL(san_francisco_loc)

In [33]:
sanfrancisco_schools.to_csv("Data/SanFrancisco/sf_schools.csv")
sanfrancisco_designing_companies.to_csv("Data/SanFrancisco/sf_design.csv")
sanfrancisco_tech_startups.to_csv("Data/SanFrancisco/sf_techstartups.csv")
sanfrancisco_leisure.to_csv("Data/SanFrancisco/sf_leisure.csv")
sanfrancisco_travelling_stations.to_csv("Data/SanFrancisco/sf_travelling.csv")
sanfrancisco_parks.to_csv("Data/SanFrancisco/sf_parks.csv")
sanfrancisco_df_entero.to_csv("Data/SanFrancisco/sf_entero.csv")

In [34]:
sanfrancisco_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
194,Bloc.io,Technological Startups,Tech Startup,US,110 Sutter St (Montgomery St),37.790252,-122.402287,"{'type': 'Point', 'coordinates': (37.790251927..."
150,Placements.io HQ,Technological Startups,Tech Startup,US,1160 Battery St Ste 40 (Union St),37.801163,-122.401514,"{'type': 'Point', 'coordinates': (37.801163031..."
356,Washington Square Park,Parks,Park,US,Columbus Ave (btwn Filbert & Union St),37.800748,-122.409786,"{'type': 'Point', 'coordinates': (37.800747642..."
39,Musée Mécanique,School,Museum,US,"Pier 45, Shed A (at Taylor St)",37.809333,-122.415952,"{'type': 'Point', 'coordinates': (37.809333068..."
201,"Affirm, Inc.",Technological Startups,Tech Startup,US,650 California St (None),37.790991,-122.400675,"{'type': 'Point', 'coordinates': (37.790991258..."
311,Muni Stop 14501,Travelling Stations,Light Rail Station,US,Embarcadero (Bay),37.806946,-122.406275,"{'type': 'Point', 'coordinates': (37.806946, -..."
322,California Cable Car Turnaround-West,Travelling Stations,Light Rail Station,US,California (Van Ness),37.7904,-122.42212,"{'type': 'Point', 'coordinates': (37.7904, -12..."
382,Parklet at Tony's Pizza,Parks,Park,US,"1570 Stockton St, San Francisco, CA 94133",37.800312,-122.409058,"{'type': 'Point', 'coordinates': (37.800312042..."
199,Pinchit,Technological Startups,Tech Startup,US,111 Pine,37.792335,-122.399504,"{'type': 'Point', 'coordinates': (37.792335324..."
119,Chum's Design & Print,Designing Studios,Print Shop,US,582 Market St Ste 100 (at 2nd St.),37.789086,-122.400448,"{'type': 'Point', 'coordinates': (37.789085904..."
